## Import Libraries

In [ ]:
import os, sys
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

from src.congestion_mining.sumo_network import SUMONetwork
from src.congestion_mining.sumo_data import *

from src.simulation.configuration_files import *
from src.simulation.demand import *
from src.simulation.additional_files import AdditionalFile

## SUMO Net

The first step of running the sumo simulation is to build the sumo network using the SUMONetwork class from sumo_network.py. This class takes path and net_file as arguments.

Note that this does not create a .net.xml file, it is assumed you have a net file. This code will help in defining parents of nodes to be used later when building the Dynamic Bayesian Network. 

#### Arguments:

1. path: this variable represents the relative path on your computer where all of your files related to the simulation will be stored

2. net_file: this is the name of the file which contains information about the network structure such as edges and nodes. This must be a net.xml file

3. additional_files: this is another .xml file that gives the simulation some additional information such as where to save the output data, the output format, and the frequency with which to gather the data

4. b: the beginning time of the simulation in seconds

5. e: the end time of the simulation in seconds

The cell below is an example. Insert your own path name and net file to begin.

In [ ]:
path="data/Sumo/huawei/simple/"
net_file="simple.net.xml"
additional_files="additional.add.xml"
b="0"
e="28800"

In [ ]:
net = SUMONetwork(path, net_file)

## Demand Profiles

Once the network is defined, you will need to define the additional file and the type of demand you would like to create in the simulation.

## Additional File

This step is crucial, as it tells sumo how to collect the data during simulation and where to save it.

#### Arguments:

1. id: type of data output
2. file: name of folder/file where to save the output data of the simulation
3. freq: frequency of data collection in seconds
4. track vehicles: leave this as True

The following code will first create the additional.add.xml file required for the simulation. You will need different additional files for each simulation so that you don't save over edge data with the latest run. This additional file code only needs to be run once, then you can manually modify the "file" attribute in the additional.add.xml file before each simulation run. Make sure the name of the additional file matches the additional_files variable you defined earlier.

In [ ]:
edgeData_attrib={"id": "edge_output",
                 "file":'random_demand/edgeData.xml',
                 "freq": "300",
                 "trackVehicles": "true"}

additional = AdditionalFile(net=net, edgeData_attrib=edgeData_attrib)

## Route Files

This step defines the demand profile which is used by duarouter in SUMO to create a .rou.xml route file. Of course, this step can be skipped if you already have your own route and trip files for your network.

### Random Demand

Add random number of vehicles

FringeFactor: how often a vehicle travels from one fringe of the network to another
Period: how often to add a new vehicle to the network in seconds

In [ ]:
random_demand = RandomDemand(net, b, e)
random_demand.run(fringeFactor="1", period="1.5")

### Static Demand

Adds vehicles to maintain demand at a static value (numberModel)

In [ ]:
static_demand = NetDemand(net, b, e, flow_model="static", numberModel=3600)
static_demand.run()

### Linear Demand

Maintain a specific demand value at specific times.

For time=[0, 9600, 19200, 28800], numberModel=[2000, 3000] the model will have a demand of 2000 vehicles from 0s to 9600s, then it will increase linearly from 2000 to 3000 vehicles starting at 9600s to 19200s, then finally it will maintain a demand of 3000 vehicles from 19200s to 28800s.

In [ ]:
linear_demand = NetDemand(net, b, e, flow_model="linear", time=[0, 9600, 19200, 28800], numberModel=[2000, 3000])
linear_demand.run()

### Wave Demand

There is no exact science to defining the curve for the wave demand. So I have some code which will plot what the curve will look like. You can play around with the values of curve1 and curve2 until you find the profile that fits your desired parameters.

In [ ]:
def wave_depart(offset, curves, t):
    v = offset
    for c in curves:
        dt = t - c[3]
        v = v + \
            c[0] * math.sin(2 * math.pi * dt * c[2]) + \
            c[1] * math.cos(2 * math.pi * dt * c[2])
        v = v / 3600.
    return v

In [ ]:
curve1 = [3000, 0, 0.0025, 0]
curve2 = [4000, 0, 0.0001389, 0]

wave_output = []
for i in range(0, 7200):
    v = wave_depart(0, [curve2], i)
    wave_output.append(v)

plt.figure(figsize=(15,5))
plt.plot(wave_output)

In [ ]:
curves = [[3000, 0, 0.0025, 0], [4000, 0, 0.00035, 0]]
wave_demand = NetDemand(net, b, e, flow_model="wave", offset=0, curves=curves)

## SUMO Configuration File

Once you have your .rou.xml files defined you can create a sumo configuration file to tell sumo which files to grab and other details about the simulation. This is the most important file, as it will be required to run the simulation for you automatically. Otherwise, you would need to find all the necessary files and add them in your SUMO session manually to run.

Make sure route_file matches with the name of the route file created in the previous step.

In [ ]:
route_file="random.rou.xml"

SimulationConfig(net, route_file, additional_files, b, e).run()

Once you have the net file, route file, additional files, and sumo configuration files you can run the simulation and gather your data.

## Next Step...

See the Root Cause Congestion Analysis notebook for data processing, DBN creation, and querying!